<a href="https://colab.research.google.com/github/IgnacioAntonio82/PROCESAMIENTO-DE-HABLA-/blob/main/TRABAJO_PRACTICO_1_Aguilar_Ignacio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ================================
# 📥 DESCARGA DEL ARCHIVO PDF
# ================================

In [32]:
import requests

#Esta función tiene como objetivo descargar un archivo PDF desde una URL y guardarlo localmente con el nombre que le indiques
def download_pdf(url, filename):
  try:
    response = requests.get(url, stream=True) # iniciar una solicitud HTTP GET hacia la URL
    response.raise_for_status() #lanza un error si la respuesta HTTP indica un problema (como error 404 si el archivo no existe).

    with open(filename, 'wb') as file:  #Abre un archivo local en modo binario de escritura ('wb').
      for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk) #Luego, lee el contenido descargado en fragmentos (chunk_size=8192 bytes, es decir, 8 KB) y lo va escribiendo en el archivo local para no saturar la memoria
    print(f"✅ PDF descargado con éxito como '{filename}'")

  except requests.exceptions.RequestException as e:
    print(f"❌ Error al descargar el PDF: {e}")


# ================================
# 📚 INSTALACIÓN DE DEPENDENCIAS ,libreria  PyMupdf
# ================================

In [33]:
!pip install pymupdf

# ================================
# 📄 PROCESAMIENTO DEL PDF
# ================================

In [34]:
import fitz  # PyMuPDF

# La función lee un archivo PDF, extrae el texto desde una página específica, y organiza ese texto en estrofas (grupos de versos), según ciertas reglas para determinar cuándo termina una estrofa
def extraer_estrofas_desde_pdf(ruta_pdf, pagina_inicio=4, max_versos_por_estrofa=8):
  ##Pagina Inicio:indica que la primer pagina a procesar es a partir de la cuarta
  ##max_versos_por_estrofa Máximo número de versos por estrofa (por defecto, 8).


    def termina_estrofa(verso):
      verso = verso.strip()
      if verso.endswith('...'):
          return False
      signos_fin = ['.', '"', '”', '?', 'zas!','mor!','rte!','aba!','sto!','ial!','jas!','sca!','era!','ajo!','nto!','mar!','nte!','vao!','los!','ros!']
      return any(verso.endswith(signo) for signo in signos_fin)
      ## esta funcion Determina el final de un verso.
      ## Considera que un verso no termina una estrofa si acaba en ... (porque se entiende que continúa).


    doc = fitz.open(ruta_pdf) #abre el archivo pdf

    texto = ""


    for pagina in doc.pages(pagina_inicio - 1, len(doc)):  #Lee todo el texto desde la página indicada (pagina_inicio - 1 porque los índices empiezan en 0).
        texto += pagina.get_text() + "\n" #asigna a texto todo el contenido en una sola cadena


    doc.close() #cierra el archivo

    lineas = [linea.strip() for linea in texto.split('\n') if linea.strip() ]
    #La variable lineas contendrá una lista con todas las líneas del texto, sin líneas vacías ni espacios sobrantes



    lista_estrofas = []
    estrofa_actual = []

    #Este código agrupa líneas de texto en estrofas según dos condiciones:
    #Cuando una línea indica que la estrofa termina (según la función termina_estrofa).
    # O cuando se alcanza un número máximo de versos por estrofa (max_versos_por_estrofa).
    #Cada estrofa se guarda como un diccionario con:
      # - La cantidad de versos que tiene (cant_versos).
      #- El texto completo de la estrofa, con los versos unidos por saltos de línea (texto).
    for linea in lineas:
        estrofa_actual.append(linea)

        if termina_estrofa(linea) or len(estrofa_actual) >= max_versos_por_estrofa:
            lista_estrofas.append({
                'cant_versos': len(estrofa_actual),
                'texto': '\n'.join(estrofa_actual)
            })
            estrofa_actual = []

    # agrega ultima estrofa si quedo alguna suelta
    if estrofa_actual:
        lista_estrofas.append({
            'cant_versos': len(estrofa_actual),
            'texto': '\n'.join(estrofa_actual)
        })

    return lista_estrofas


# ================================
# ▶️ EJECUCIÓN PRINCIPAL
# ================================

In [37]:
def main():
    pdf_url = "https://www.argentina.gob.ar/sites/default/files/hernandez_jose_-_el_gaucho_martin_fierro.pdf"
    output_filename = "martin_fierro.pdf"

    download_pdf(pdf_url, output_filename)
    estrofas = extraer_estrofas_desde_pdf(output_filename)

    # Muestra la  estrofa como ejemplo
    print("\n📝 Estrofa extraída:\n")
    print(estrofas[1])

main()

✅ PDF descargado con éxito como 'martin_fierro.pdf'

📝 Estrofa extraída:

{'cant_versos': 6, 'texto': 'Pido a los santos del cielo\nque ayuden mi pensamiento:\nles pido en este momento\nque voy a cantar mi historia\nme refresquen la memoria\ny aclaren mi entendimiento.'}
